In [1]:
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [3]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True  
session = tf.Session(config=config)
 
KTF.set_session(session)

Using TensorFlow backend.


In [4]:
import numpy as np
import keras_bert

In [5]:
from hypers import *
from preprocess import *
from Knowledge import *
from Models import *

In [6]:
bert_path='/data1/qitao/BERT/chinese_L-12_H-768_A-12/'
emb_root_path = '/data/data/qit/cnchar.model'

# Load BERT Model

In [7]:
bert_word_index_path=bert_path+'vocab.txt'

bert_config_path = os.path.join(bert_path,'bert_config.json')
bert_checkpoint_path = os.path.join(bert_path,'bert_model.ckpt')
bert_vocab_path = os.path.join(bert_path,'vocab.txt')

bert_voca = read_bert_word_index(bert_word_index_path)

bert = keras_bert.load_trained_model_from_checkpoint(bert_config_path,
                                                        bert_checkpoint_path, 
                                                        training=False,
                                                        trainable=True,
                                                        seq_len=max_text_length,
                                                        )

# Load and Preprocess Data

In [8]:
TrainData,TestData,Centorid = read_data()

In [9]:
char_dict = {}
tp_dict = {}
    
train_input, train_label, train_mask = parse_data(TrainData,Centorid,char_dict,tp_dict,bert_voca)
test_input, test_label, test_mask = parse_data(TestData,Centorid,char_dict,tp_dict,bert_voca)
train_mask = train_mask.reshape((train_mask.shape[0],max_entity_num*max_entity_num,))
train_label = train_label.reshape((train_label.shape[0],max_entity_num*max_entity_num,1))

/home/qit16/KeMRE-Release/Knowledge.py:91: RuntimeWarning: invalid value encountered in double_scalars
  sim = sim/(np.sqrt(sim_length)*np.sqrt(len(cen_contexts)))


In [10]:
emb_matrix=Init_char_embedding(emb_root_path,char_dict)

# Model Training

In [ ]:
use_knowledge=True
use_bert = True
char_cnn = True
char_lstm = True
entity_type = True
entity_cnn = True
entity_lstm = True
dim_knowledge = 150

model = get_model(emb_matrix,bert,tp_dict,use_knowledge,use_bert,char_cnn,char_lstm,entity_type,entity_cnn,entity_lstm,dim_knowledge)
model.compile(Adam(0.0001),loss='binary_crossentropy',sample_weight_mode="temporal")

for i in range(25):
    model.fit(train_input,train_label,epochs = 5,batch_size=2,sample_weight=train_mask)
    pred = infer(model,test_input,test_mask)
    g = evaluation(pred,test_label,test_mask)
    print('test',g)

Epoch 1/5
242/242 [==============================] - 352s 1s/step - loss: 0.1258
Epoch 2/5
242/242 [==============================] - 324s 1s/step - loss: 0.0395
Epoch 3/5
242/242 [==============================] - 320s 1s/step - loss: 0.0124
Epoch 4/5
242/242 [==============================] - 314s 1s/step - loss: 0.0062
Epoch 5/5
242/242 [==============================] - 319s 1s/step - loss: 0.0052
test (0.9979094917468203, 0.9294918997671675, 0.9969394806005588, 0.9620349639439781)
Epoch 1/5
242/242 [==============================] - 325s 1s/step - loss: 0.0045
Epoch 2/5
242/242 [==============================] - 324s 1s/step - loss: 0.0035
Epoch 3/5
242/242 [==============================] - 317s 1s/step - loss: 0.0035
Epoch 4/5
242/242 [==============================] - 318s 1s/step - loss: 0.0030
Epoch 5/5
242/242 [==============================] - 315s 1s/step - loss: 0.0030
test (0.9994780286795699, 0.9846281295117242, 0.9970174293040341, 0.9907840502949304)
Epoch 1/5
242/242 